In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Load data sets
(x_train_all, y_train_all), (x_test, y_test)= tf.keras.datasets.imdb.load_data(skip_top=20, num_words=100)


17465344/17464789 [==============================] - 0s 0us/step


skip top은 건너뛸 단어의 개수 지정
num_word : 훈련에 사용될 단어 수 

In [3]:
print(x_train_all.shape, y_train_all.shape)

(25000,) (25000,)


In [4]:
# bag of word
print(x_train_all[0])

[2, 2, 22, 2, 43, 2, 2, 2, 2, 65, 2, 2, 66, 2, 2, 2, 36, 2, 2, 25, 2, 43, 2, 2, 50, 2, 2, 2, 35, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 39, 2, 2, 2, 2, 2, 2, 38, 2, 2, 2, 2, 50, 2, 2, 2, 2, 2, 2, 22, 2, 2, 2, 2, 2, 22, 71, 87, 2, 2, 43, 2, 38, 76, 2, 2, 2, 2, 22, 2, 2, 2, 2, 2, 2, 2, 2, 2, 62, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 66, 2, 33, 2, 2, 2, 2, 38, 2, 2, 25, 2, 51, 36, 2, 48, 25, 2, 33, 2, 22, 2, 2, 28, 77, 52, 2, 2, 2, 2, 82, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 36, 71, 43, 2, 2, 26, 2, 2, 46, 2, 2, 2, 2, 2, 2, 88, 2, 2, 2, 2, 98, 32, 2, 56, 26, 2, 2, 2, 2, 2, 2, 2, 22, 21, 2, 2, 26, 2, 2, 2, 30, 2, 2, 51, 36, 28, 2, 92, 25, 2, 2, 2, 65, 2, 38, 2, 88, 2, 2, 2, 2, 2, 2, 2, 2, 32, 2, 2, 2, 2, 2, 32]


In [5]:
# except 0, 1, 2
for i in range(len(x_train_all)):
    x_train_all[i] = [w for w in x_train_all[i] if w>2]

print(x_train_all[0])

[22, 43, 65, 66, 36, 25, 43, 50, 35, 39, 38, 50, 22, 22, 71, 87, 43, 38, 76, 22, 62, 66, 33, 38, 25, 51, 36, 48, 25, 33, 22, 28, 77, 52, 82, 36, 71, 43, 26, 46, 88, 98, 32, 56, 26, 22, 21, 26, 30, 51, 36, 28, 92, 25, 65, 38, 88, 32, 32]


In [6]:
# down load word index
word_to_index = tf.keras.datasets.imdb.get_word_index()
word_to_index['movie']

1646592/1641221 [==============================] - 0s 0us/step


17

In [7]:
# 훈련셋을 영단어로 변환
index_to_word = {word_to_index[k]: k for k in word_to_index}

for w in x_train_all[0]:
    print(index_to_word[w-3], end=' ')

film just story really they you just there an from so there film film were great just so much film would really at so you what they if you at film have been good also they were just are out because them all up are film but are be what they have don't you story so because all all 

In [9]:
print(len(x_train_all[0]), len(x_train_all[1]))
# 샘플의 길이가 다르다

59 32


In [0]:
# divide sample
np.random.seed(42)
random_index = np.random.permutation(25000)

x_train = x_train_all[random_index[:20000]]
y_train = y_train_all[random_index[:20000]]
x_val = x_train_all[random_index[20000:]]
y_val = y_train_all[random_index[20000:]]

In [0]:
# sample length fitting
maxlen = 100
x_train_seq = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
y_val_seq = tf.keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

In [0]:
# sample one-hot-encoding
x_train_onehot = tf.keras.utils.to_categorical(x_train_seq)
x_val_onehot = tf.keras.utils.to_categorical(y_val_seq)

In [0]:
# ㅇ우워원ㅎ하핫 ㅇ이인ㅋ코콛코디딩ㅇ으은 ㅁ마만많ㅇ으은 ㄷ데뎅데이잍이터털터르를 ㅍ피필ㅇ요욜요로 ㅎ하한ㄷ다.
class RNN:
    def __init__(self, n_cells=10, batch_size=32, learning_rate=0.1):
        self.n_cells = n_cells
        self.batch_size = batch_size
        self.w1h = None  # 은닉 상태 가중치
        self.w1x = None  # 입력 가중치
        self.b1 = None
        self.w2 = None  # 출력층 가중치와 절편
        self.b2 = None
        self.h = None
        self.losses = []  # 훈련 손실
        self.val_losses = []  # 검증 손실
        self.lr = learning_rate

    def init_weight(self, n_features, n_classes):
        # initialize weight
        orth = tf.initializers.Orthogonal()
        glorot = tf.initializers.GlorotUniform()

        self.w1h = orth((self.n_cells, self.n_cells)).numpy()
        self.w1x = glorot((n_features, self.n_cells)).numpy()
        self.b1 = np.zeros(self.n_cells)
        self.w2 = glorot((self.n_cells, n_classes)).numpy()
        self.b2 = np.zeros(n_classes)

    def forpass(self, x):
        self.h = [np.zeros((x.shape[0], self.n_cells))]  # 은닉 상태 초기화
        seq = np.swapaxes(x, 0, 1)  # 배치 차원과 스텝 차원을 바꾸기
        for z in seq:
            z1 = np.dot(z, self.w1x) + np.dot(self.h[-1], self.w1h) + self.b1
            h = np.tanh(z1)
            self.h.append(h)
            z2 = np.dot(h, self.w2) + self.b2
        return z2

    def backprop(self, x, err):
        m = len(x)
        w2_grad = np.dot(self.h[-1].T, err) / m
        b2_grad = np.sum(err)/ m

        seq = np.swapaxes(x, 0, 1)
        w1h_grad = w1x_grad = b1_grad = 0
        # 셀 직전까지의 에러 계산
        err_to_cell = np.dot(err, self.w2.T) * (1-self.h[-1]**2)
        for x, h in zip(seq[::1][:10], self.h[:-1][::-1][:10]):
            w1h_grad += np.dot(h.T, err_to_cell)
            w1x_grad += np.dot(x.T, err_to_cell)
            b1_grad += np.sum(err_to_cell, axis=0)
            err_to_cell = np.dot(err_to_cell, self.w1h)*(1-h**2)

        w1h_grad /= m
        w1x_grad /= m
        b1_grad /= m

        return w1h_grad, w1x_grad, b1_grad, w2_grad, b2_grad